In [1]:
# Import packages 
import pandas as pd
import numpy as np
# Set Working Directory
import os
# os.chdir(r'C:\Personal\IMM') # absolute path, using \ and r prefix
# wd = os.getcwd()

import json
from pprint import pprint
import timeit
from decimal import *
import numba
from numba import jit
import traceback

import itertools

import functools
from balsa.matrices import read_mdf, to_mdf, to_fortran, read_fortran_square, read_fortran_rectangle

seed= 12345

# Directory Listing

In [2]:
dirListing = 'c:\\personal\\IMM'
dirListing1 = 'c:\\personal\\IMM\\Offpeak'

dirListing_other_pk = 'c:\\personal\\IMM\\Other Trips\\peak'
fpath_pk = os.listdir(dirListing_other_pk)

dirListing_other_offpk = 'c:\\personal\\IMM\\Other Trips\\offpeak'
fpath_offpk = os.listdir(dirListing_other_offpk)

#### Bring in the ggh zone definitions

In [8]:
# batch in ggh zone numbers and add in two columns for i and j zones
ggh = pd.read_csv(os.path.join(dirListing, "GGH_zones.csv"))
if 'ggh_zone'  not in ggh:
    print("The 'ggh_zone' column does not exist.")

# Batch in the Other Trips i.e. non-hbw/hbs/hbu

In [9]:
# Now append the peak csv file names
BinaryMat_other_pk = []
for item in fpath_pk:
    if item.endswith(".bin"):
        fname = item
        fname = fname.split('.')[0]  # get the name only
        BinaryMat_other_pk.append(fname)

# Now append the off peak csv file names
BinaryMat_other_offpk = []
for item in fpath_offpk:
    if item.endswith(".bin"):
        fname = item
        fname = fname.split('.')[0]  # get the name only
        BinaryMat_other_offpk.append(fname)

# Bring in the Trips and household file

In [10]:
# set dictionary of Dtypes
dtype_trips = {}
dtype_hhold = {}

# dictionary for storing hhold column dtypes
dtype_hhold = {'hhid':'int32',
              'taz':'int16',
              'hhinc':'int32',
              'dtype':'int8',
              'hhsize':'int8',
              'nveh':'int8',
              'auto_suff': 'int8',
              'segment': 'int8',
              'segment1': 'int8'}

# dictionary for storing trips column dtypes
dtype_trips = {'hhid':'int32',
              'pid':'int8',
              'tour_id':'int8',
              'subtour_id':'int8',
              'trip_id':'int8',
              'activity_i':'category',
              'activity_j':'category',
              'taz_i': 'int16',
              'taz_j': 'int16',
              'tour_direction':'category',
              'purpose': 'category',
              'trip_direction': 'category',
              'peak_factor': 'float64'}


In [11]:
%%time
trips = pd.read_csv(r"c:\personal\IMM\trips_out.csv")
hhold = pd.read_csv(r"c:\personal\IMM\households_out.csv")

Wall time: 46 s


In [12]:
%%time
# Add in market segment definition. We need two segments - one that defines the 6
# and second that is a dummy for use in the univ, school, NHB
hhold.loc[(hhold['hhinc'] <= 60000) & (hhold['auto_suff'] == 0), 'segment'] = 1
hhold.loc[(hhold['hhinc'] > 60000) & (hhold['auto_suff'] == 0), 'segment'] = 2
hhold.loc[(hhold['hhinc'] <= 60000) & (hhold['auto_suff'] == 1), 'segment'] = 3
hhold.loc[(hhold['hhinc'] > 60000) & (hhold['auto_suff'] == 1), 'segment'] = 4
hhold.loc[(hhold['hhinc'] <= 60000) & (hhold['auto_suff'] == 2), 'segment'] = 5
hhold.loc[(hhold['hhinc'] > 60000) & (hhold['auto_suff'] == 2), 'segment'] = 6

hhold['segment1'] = 1

Wall time: 437 ms


In [13]:
%%time
# set dtypes for trips df
for key, value in dtype_trips.items():
    trips[key] = trips[key].astype(value)

# set dtypes for hhold df
for key, value in dtype_hhold.items():
    hhold[key] = hhold[key].astype(value)

Wall time: 10.1 s


# Build the peak and non-peak trips dataframe dictionary

### Functions

In [14]:
### Function to convert from bin to dataframe and set indices. Also unstack the dataframe and rename columns
def convert_df(location, name, nzones):
    '''
    
    '''
    # read in the fortran dataframe and then subset it for the internal zones
    # in the GGH.
    df = read_fortran_rectangle(os.path.join(location, name + ".bin"), n_columns = 4000, tall = False, reindex_rows = False, fill_value = None)
    df1 = pd.DataFrame(df).iloc[:nzones, :nzones]
    
    # set column and row indices
    df1.rename(columns = ggh['ggh_zone'], inplace = True )
    df1.set_index(ggh['ggh_zone'], inplace = True)
    
    # Now unstack and rename columns
    df1 = df1.unstack().reset_index()
    df1.columns = ['origin', 'destination', 'trips']
    
    # Remove zero trips
    df1 = df1.loc[df1['trips'] != 0]
    
    return(df1)

In [15]:
# set dictionary of Dtypes
df_trips_structure = {}

# dictionary for storing column dtypes
df_trips_structure = {'origin':'int16',
              'destination':'int16',
              'trips':'float32',
              'period': 'category'}

In [16]:
%%time
## peak period dataframe for Other Trips
other_pk = {}
other_offpk = {}

for name in BinaryMat_other_pk:
    other_pk[name] = convert_df(dirListing_other_pk, name, 3262)
    other_pk[name]['period'] = 'peak'
    
    # reset column types
    for key, value in df_trips_structure.items():
        other_pk[name][key] = other_pk[name][key].astype(value)

for name in BinaryMat_other_offpk:
    other_offpk[name] = convert_df(dirListing_other_offpk, name, 3262)
    other_offpk[name]['period'] = 'offpeak'
    
    # reset column types
    for key, value in df_trips_structure.items():
        other_offpk[name][key] = other_offpk[name][key].astype(value)
    

Wall time: 1min 27s


In [17]:
df = other_pk['trips_peak_all_modes_hbm_insuff_high']
#df.columns = ['origin', 'destination', 'trips']
len(df.loc[df['trips'] == 0])

0

In [18]:
for key, value in other_pk.items():
    print(key)

trips_peak_all_modes_hbm_insuff_high
trips_peak_all_modes_hbm_insuff_low
trips_peak_all_modes_hbm_nocar_high
trips_peak_all_modes_hbm_nocar_low
trips_peak_all_modes_hbm_suff_high
trips_peak_all_modes_hbm_suff_low
trips_peak_all_modes_hbo_insuff_high
trips_peak_all_modes_hbo_insuff_low
trips_peak_all_modes_hbo_nocar_high
trips_peak_all_modes_hbo_nocar_low
trips_peak_all_modes_hbo_suff_high
trips_peak_all_modes_hbo_suff_low
trips_peak_all_modes_nhb_all_segments
trips_peak_all_modes_wbo_insuff_high
trips_peak_all_modes_wbo_insuff_low
trips_peak_all_modes_wbo_nocar_high
trips_peak_all_modes_wbo_nocar_low
trips_peak_all_modes_wbo_suff_high
trips_peak_all_modes_wbo_suff_low


In [ ]:
for k,v in other_pk.items():
    print(len(v.groupby(['origin', 'destination']).size()))

# Bucket round

In [14]:
def bucket_rounding(df, newround_f, residual_f):
    """
    This function bucket rounds a dataframe given a set of values in a column.
    
    Arguments: dataframe, first rounded value, and first residual
    """
    newround_f.append((df['trips'].values[i] + residual_f[i-1]).round())
    residual_f.append(df['trips'].values[i] + residual_f[i-1] - newround_f[i]) 
    
    return (newround_f)

In [35]:
#other_pk1 = dict(list(other_pk.items())[:1])

### Constrained over the entire dataframe

#### Peak Period

In [50]:
%%time
# collect the dataframes
finaldf_other_pk = {}

# run FOR loop for each of the dataframes in the dictionary
for name in other_pk.keys():
        
    # get df from dictionary
    finaldf_other_pk[name] = other_pk[name]
      
    #create empty lists
    newround_pk_f = []
    newround_pk_s = []
    residual_pk_f = []
    residual_pk_s = []
        
    # get the first row values
    newround_pk_f = [finaldf_other_pk[name].iat[0,2].round()]
    residual_pk_f = [finaldf_other_pk[name].iat[0,2] - newround_pk_f[0]]
    
    for i in range(1, len(finaldf_other_pk[name].index)):
                            
        # carry out bucket rounding
        bucket_rounding(finaldf_other_pk[name], newround_pk_f, residual_pk_f)           
    
    # cbind the final trips and only keep rows greater than zero.
    finaldf_other_pk[name]['finaltrips'] = newround_pk_f


Wall time: 19min 8s


In [51]:
for k, v in finaldf_other_pk.items():
    print(repr(finaldf_other_pk[k]['trips'].sum()), finaldf_other_pk[k]['finaltrips'].sum())

188397.98 188398.0
79616.758 79617.0
18276.752 18277.0
50443.402 50443.0
172711.42 172712.0
110728.1 110728.0
406142.56 406143.0
161784.52 161785.0
40770.273 40770.0
107369.13 107369.0
379535.31 379535.0
224161.11 224161.0
933184.5 933185.0
728631.88 728632.0
186148.64 186149.0
61102.043 61102.0
62906.723 62907.0
670073.19 670073.0
252038.36 252038.0


#### Off-peak period

In [54]:
%%time
# collect the dataframes
finaldf_other_offpk = {}

# run FOR loop for each of the dataframes in the dictionary
for name in other_offpk.keys():
        
    # get df from dictionary
    finaldf_other_offpk[name] = other_offpk[name]
    
    #create empty lists
    newround_pk_f = []
    newround_pk_s = []
    residual_pk_f = []
    residual_pk_s = []
        
    # get the first row values
    newround_pk_f = [finaldf_other_offpk[name].iat[0,2].round()]
    residual_pk_f = [finaldf_other_offpk[name].iat[0,2] - newround_pk_f[0]]
    
    for i in range(1, len(finaldf_other_offpk[name].index)):
                            
        # carry out bucket rounding
        bucket_rounding(finaldf_other_offpk[name], newround_pk_f, residual_pk_f)           
    
    # cbind the final trips and only keep rows greater than zero.
    finaldf_other_offpk[name]['finaltrips'] = newround_pk_f

Wall time: 17min 19s


In [55]:
for k, v in finaldf_other_offpk.items():
    print(repr(finaldf_other_offpk[k]['trips'].sum()), finaldf_other_offpk[k]['finaltrips'].sum())

354075.0 354075.0
167898.19 167898.0
36202.254 36202.0
116856.59 116857.0
318137.47 318137.0
232781.91 232782.0
688581.63 688581.0
274292.41 274292.0
59954.723 59955.0
164714.03 164714.0
643470.63 643471.0
380047.13 380047.0
1310048.8 1310048.0
479712.84 479713.0
122555.43 122555.0
40227.992 40228.0
41416.25 41416.0
441158.72 441159.0
165935.7 165936.0


In [56]:
t = finaldf_other_offpk['trips_offpeak_all_modes_hbm_insuff_high']
t.head()

,origin,destination,trips,period,finaltrips
0,1001,1001,12.490139,offpeak,12.0
1,1001,1002,17.429909,offpeak,18.0
2,1001,1003,3.205833,offpeak,3.0
3,1001,1004,3.349331,offpeak,3.0
4,1001,1005,0.221551,offpeak,1.0


In [43]:
t['finaltrips'].sum()
len(t)

4883855

In [44]:
t1 = t.loc[t['finaltrips'] > 0]
t1['finaltrips'].sum()
len(t1)

82798

In [ ]:
### Function that acts upon each of the df in the folder
def expand_df(dfrepeat, colsrepeat):
    '''
    This function prepares every dataframe in the folder repeating the dataframe \
    rows using a user defined column. 
    
    Arguments: Dataframe and column that contains value to repeat rows
    
    Return: expanded dataframe 
    
    '''
    if (dfrepeat[colsrepeat].dtype == np.float64):
        dfrepeat[colsrepeat] = dfrepeat[colsrepeat].astype(int)
        df1 = dfrepeat.loc[np.repeat(dfrepeat.index.values, dfrepeat[colsrepeat])]
            
    return(df1)

In [ ]:
%%time
# Merge the hholds info to the trips. By doing so, we can bring in a bunch of household attributes
# including income, dwelling type, size, number of vehicles, and auto_sufficiency. Add in an integer 
# definition for one of six market segments.
trips_hhold = pd.merge(trips, hh, how = 'left', left_on = 'hhid', right_on = 'hhid')

In [ ]:
dirListing1 = 'c:\\personal\\IMM\\Other Trips\\offpeak'
hbm = read_fortran_rectangle(os.path.join(dirListing1, "trips_offpeak_all_modes_hbm_insuff_high.bin"), n_columns = 4000, tall = False, reindex_rows = False, fill_value = None)


In [ ]:
hbm1 = pd.DataFrame(hbm).iloc[:3262, :3262]
#hbm1 = hbm1.iloc[:3262, :3262]
hbm1.shape

In [ ]:
hbm1.rename(columns = ggh['ggh_zone'], inplace = True )
hbm1.set_index(ggh['ggh_zone'], inplace = True)

In [ ]:
hbm1

In [ ]:
hbm1.head()

# Now do for HBW, HBS, and HBU

In [19]:
# batch in ggh zone numbers and add in two columns for i and j zones
ggh['key'] = 0
# make a copy of the df and create square matrix
ggh1 = ggh
ggh2= pd.merge(ggh1, ggh, how='left', on = 'key')

In [22]:
trips_hhold.head()

,hhid,pid,tour_id,subtour_id,trip_id,activity_i,activity_j,taz_i,taz_j,tour_direction,...,trip_direction,peak_factor,taz,hhinc,dtype,hhsize,nveh,auto_suff,segment,segment1
0,1,0,0,-1,0,home,work,1001,1015,outbound,...,outbound,0.777,1001,110000,5,1,1,2,6,1
1,1,0,0,-1,1,work,home,1015,1001,inbound,...,inbound,0.777,1001,110000,5,1,1,2,6,1
2,2,0,0,-1,0,home,work,1001,1035,outbound,...,outbound,0.777,1001,36000,6,1,1,2,5,1
3,2,0,0,-1,1,work,home,1035,1001,inbound,...,inbound,0.777,1001,36000,6,1,1,2,5,1
4,2,0,0,0,0,work,business,1035,0,outbound,...,outbound,0.603,1001,36000,6,1,1,2,5,1


In [ ]:
#ggh = ggh.assign(taz_i = ggh['ggh_zone'], taz_j = ggh['ggh_zone'])

In [21]:
%%time
# Merge the hholds info to the trips. By doing so, we can bring in a bunch of household attributes
# including income, dwelling type, size, number of vehicles, and auto_sufficiency. Add in an integer 
# definition for one of six market segments.
trips_hhold = pd.merge(trips, hhold, how = 'left', left_on = 'hhid', right_on = 'hhid')

Wall time: 7.66 s


In [ ]:
%%time

# only keep the HBW, HBU, and HBS trip purposes
trips_hhold = trips_hhold.loc[(trips_hhold['purpose'] == 'HBW') | (trips_hhold['purpose'] == 'HBU') | (trips_hhold['purpose'] == 'HBS')]

# create the six dataframes that Bill needs to get probabilities from MLogit. 
# First do it for HBW.
hbw_only = trips_hhold.loc[trips_hhold['purpose'] == 'HBW']
gp_hbw = hbw_only.groupby(['purpose', 'segment'])

for name_hbw, data_hbw in gp_hbw:
    # create filename and then groupby
    # only keep relevant cols and set a flag
    # Merge the ggh zones and the trip list and convert to wide format
    fname = name_hbw[0] + "_" + str(name_hbw[1])
    df_hbw = data_hbw.groupby(['taz_i', 'taz_j']).size().reset_index()
    df_hbw = df_hbw[['taz_i', 'taz_j']]
    df_hbw['flag'] = 1
    df_hbw1 = pd.merge(ggh2, df_hbw, how = "left", left_on = ['ggh_zone_x', 'ggh_zone_y'], right_on = ['taz_i', 'taz_j'])
    df_hbw2 = df_hbw1.pivot_table(index = 'ggh_zone_x', columns = 'ggh_zone_y', values = 'flag', fill_value = 0)
    
    to_fortran(df_hbw2, os.path.join(dirListing, fname + '.bin'), n_columns = 4000)

# Now do it for HBU and HBS
ed_only = trips_hhold.loc[(trips_hhold['purpose'] == 'HBS') | (trips_hhold['purpose'] == 'HBU')]
gp_ed = ed_only.groupby(['purpose', 'segment1'])


# for name, data_SchUniv in gp_ed:
#     # create filename and then groupby
#     # only keep relevant cols and set a flag
#     # Merge the ggh zones and the trip list and convert to wide format
#     fname = name[0] + "_" + str(name[1])
#     df = data_SchUniv.groupby(['taz_i', 'taz_j']).size().reset_index()
#     df = df[['taz_i', 'taz_j']]
#     df['flag'] = 1
#     df1 = pd.merge(ggh2, df, how = "left", left_on = ['ggh_zone_x', 'ggh_zone_y'], right_on = ['taz_i', 'taz_j'])
#     df2 = df1.pivot_table(index = 'ggh_zone_x', columns = 'ggh_zone_y', values = 'flag', fill_value = 0)
    
#     to_fortran(df2, os.path.join(dirListing, fname + '.bin'), n_columns = 4000)

In [ ]:
df2.index.equals(df2.columns)

In [ ]:
df2.shape

In [ ]:

to_fortran(df2, os.path.join(dirListing, fname + '.csv'), n_columns=4000)

In [ ]:
to_fortran?